In [1]:
!pip install pygame


   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 5.6 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.6 MB 15.7 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 16.2 MB/s eta 0:0

In [3]:
!pip install torch torchvision torchaudio


   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.3/204.1 MB 7.5 MB/s eta 0:00:28
    --------------------------------------- 3.4/204.1 MB 8.8 MB/s eta 0:00:23
   - -------------------------------------- 6.0/204.1 MB 10.3 MB/s eta 0:00:20
   - -------------------------------------- 8.9/204.1 MB 11.1 MB/s eta 0:00:18
   -- ------------------------------------- 12.1/204.1 MB 12.0 MB/s eta 0:00:17
   --- ------------------------------------ 15.5/204.1 MB 12.8 MB/s eta 0:00:15
   --- ------------------------------------ 19.4/204.1 MB 13.6 MB/s eta 0:00:14
   ---- ----------------------------------- 23.6/204.1 MB 14.6 MB/s eta 0:00:13
   ----- ---------------------------------- 28.0/204.1 MB 15.3 MB/s eta 0:00:12
   ----- ---------------------------------- 29.9/204.1 MB 15.2 MB/s eta 0:00:12
   ------ --------------------------------- 30.9/204.1 MB 14.1 MB/s eta 0:00:13
   ------ --------------------------------- 32.2/204.1 

In [4]:
import pygame
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque


In [ ]:
# Initialize pygame
pygame.init()

# Set device for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Game Constants
WIDTH, HEIGHT = 800, 600
CAR_WIDTH, CAR_HEIGHT = 40, 60
WHITE, BLACK, RED, GREEN, BLUE = (255,255,255), (0,0,0), (255,0,0), (0,255,0), (0,0,255)

# Car Class
class Car:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.vel = 5
        self.angle = 0
        self.image = pygame.Surface((CAR_WIDTH, CAR_HEIGHT))
        self.image.fill(RED)
        self.rect = self.image.get_rect(center=(self.x, self.y))
    
    def move(self, action):
        if action == 0:  # Left
            self.angle += 5
        elif action == 1:  # Right
            self.angle -= 5
        elif action == 2:  # Forward
            self.x += self.vel * np.cos(np.radians(self.angle))
            self.y -= self.vel * np.sin(np.radians(self.angle))
        elif action == 3:  # Reverse
            self.x -= self.vel * np.cos(np.radians(self.angle))
            self.y += self.vel * np.sin(np.radians(self.angle))
        
        self.rect = self.image.get_rect(center=(self.x, self.y))
    
    def draw(self, screen):
        rotated_image = pygame.transform.rotate(self.image, self.angle)
        screen.blit(rotated_image, self.rect.topleft)

# AI Model (Neural Network)
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 24).to(device)
        self.fc2 = nn.Linear(24, 24).to(device)
        self.fc3 = nn.Linear(24, output_dim).to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# AI Agent
class CarAI:
    def __init__(self):
        self.model = DQN(4, 4).to(device)  # Move model to GPU
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.memory = deque(maxlen=1000)  # Reduce memory size for faster training
        self.gamma = 0.95
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
    
    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, 3)
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            action_values = self.model(state)
            return torch.argmax(action_values).item()
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def train(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            next_state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0).to(device)
            target = torch.tensor(reward, dtype=torch.float32).to(device)
            if not done:
                target += self.gamma * torch.max(self.model(next_state)).item()
            target_f = self.model(state).clone()
            target_f[0][action] = target
            
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(self.model(state), target_f)
            loss.backward()
            self.optimizer.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def save_model(self):
        torch.save(self.model.state_dict(), "car_ai_model.pth")
    
    def load_model(self):
        self.model.load_state_dict(torch.load("car_ai_model.pth"))
        self.model.eval()

# Main Game Loop
def main():
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()
    car = Car(WIDTH // 2, HEIGHT - 100)
    ai = CarAI()
    
    frame_count = 0
    running = True
    while running:
        screen.fill(WHITE)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        # AI chooses action
        state = torch.tensor([car.x / WIDTH, car.y / HEIGHT, np.sin(np.radians(car.angle)), np.cos(np.radians(car.angle))], dtype=torch.float32).to(device)
        action = ai.choose_action(state.cpu().numpy())
        car.move(action)
        ai.remember(state.cpu().numpy(), action, -1, state.cpu().numpy(), False)
        
        # Train every 10 frames for faster execution
        if frame_count % 10 == 0:
            ai.train()
        
        car.draw(screen)
        pygame.display.flip()
        clock.tick(60)  # Increase FPS for faster training
        frame_count += 1
    
    ai.save_model()
    pygame.quit()

if __name__ == "__main__":
    main()
